In [1]:
import wfdb
import csv
import pandas as pd
import numpy as np
import os

In [2]:
import os
import csv
import pandas as pd
import wfdb  # Make sure wfdb is installed

output_dir = "data_creation"
os.makedirs(output_dir, exist_ok=True)

# Patient numbers
patient_numbers = [
    "100", "101", "102", "103", "104", "105", "106", "107", "108", "109",
    "111", "112", "113", "114", "115", "116", "117", "118", "119", "121",
    "122", "123", "124", "200", "201", "202", "203", "205", "207", "208",
    "209", "210", "212", "213", "214", "215", "217", "219", "220", "221",
    "222", "223", "228", "230", "231", "232", "233", "234"
]

# N = normal 
# S = supra-ventricular premature
# V = ventricular escape
# F = fusion of ventricular and normal
# Q = unclassified heartbeats
symbol_to_category = {
    'N': 'N', '.': 'N', 'L': 'N', 'R': 'N', 'e': 'N', 'j': 'N',
    'a': 'S', 'A': 'S', 'J': 'S', 'S': 'S',
    'V': 'V', 'E': 'V',
    'F': 'F',
    '/': 'Q', 'f': 'Q', 'Q': 'Q'
}

for patient_number in patient_numbers:
    try:
        # ECG data
        path_to_record = f"mit-database/{patient_number}"
        patient_record = wfdb.rdrecord(path_to_record)
        leads = patient_record.sig_name
        ecg_data = patient_record.p_signal

        # ECG CSV
        ecg_filename = f"{output_dir}/{patient_number}_ECG.csv"
        with open(ecg_filename, "w", newline='') as outfile:
            out_csv = csv.writer(outfile)
            out_csv.writerow(leads)
            for row in ecg_data:
                out_csv.writerow(row)

        # Annotations data
        annotation = wfdb.rdann(path_to_record, 'atr')
        symbols = annotation.symbol
        annotations = annotation.sample

        # Filter out symbols not in symbol_to_category
        filtered_symbols_annotations = [(sym, ann) for sym, ann in zip(symbols, annotations) if sym in symbol_to_category]
        categories = [symbol_to_category[sym] for sym, ann in filtered_symbols_annotations]
        annotations_filtered = [ann for sym, ann in filtered_symbols_annotations]

        df_annotations = pd.DataFrame({'Category': categories, 'Annotation': annotations_filtered})

        # Annotations CSV
        annotations_filename = f"{output_dir}/{patient_number}_Annotations.csv"
        df_annotations.to_csv(annotations_filename, index=False)

    except Exception as e:
        print(f"Failed to process: {patient_number}: {e}")

print("Done")

Done


In [3]:
import os
import pandas as pd
import numpy as np


def process_patient_data(patient_number, data_creation_dir="data_creation"):
    ecg_file_path = os.path.join(data_creation_dir, f"{patient_number}_ECG.csv")
    annotations_file_path = os.path.join(data_creation_dir, f"{patient_number}_Annotations.csv")
    
    patient_X = []
    patient_Y = []
    
    try:
        ecg_df = pd.read_csv(ecg_file_path)
        annotations_df = pd.read_csv(annotations_file_path)
    except FileNotFoundError:
        print(f"Files for patient {patient_number} not found. Skipping...")
        return [], []
    
    first_column_name = ecg_df.columns[0]
    second_column_name = ecg_df.columns[1] if len(ecg_df.columns) > 1 else None

    sampling_rate = 360  # Hz
    total_window_size_seconds = 1  # Total window size in seconds
    total_window_size_samples = total_window_size_seconds * sampling_rate

    for _, row in annotations_df.iterrows():
        annotation_point = row['Annotation']
        category = row['Category']
        
        # Randomly determine the window split around the annotation point
        before_seconds = np.random.uniform(0, total_window_size_seconds)
        after_seconds = total_window_size_seconds - before_seconds
        before_samples = int(before_seconds * sampling_rate)
        after_samples = int(after_seconds * sampling_rate)
        
        start_point = max(0, annotation_point - before_samples)
        end_point = start_point + total_window_size_samples  # Ensure the window is of the exact expected size
        
        # Adjust the end point if it exceeds the length of the data
        if end_point > len(ecg_df):
            end_point = len(ecg_df)
            start_point = max(0, end_point - total_window_size_samples)  # Adjust start point accordingly
        
        # Process data from the first column
        window_data_first_column = ecg_df.iloc[start_point:end_point][first_column_name].to_numpy()
        
        patient_X.append(window_data_first_column)
        patient_Y.append(category)
        
        # If there's a second column, process it and add as a new entry
        if second_column_name:
            window_data_second_column = ecg_df.iloc[start_point:end_point][second_column_name].to_numpy()
            
            patient_X.append(window_data_second_column)
            patient_Y.append(category)  # Repeat category for the new entry
    
    return patient_X, patient_Y



# Initialize lists to hold the entire dataset
all_X = []
all_Y = []

data_creation_dir = "data_creation"

# Process each patient
for patient_number in patient_numbers:
    patient_X, patient_Y = process_patient_data(patient_number, data_creation_dir)
    all_X.extend(patient_X)
    all_Y.extend(patient_Y)

X = np.array(all_X)
Y = np.array(all_Y)

In [4]:
X

array([[-0.145, -0.145, -0.145, ..., -0.445, -0.475, -0.51 ],
       [-0.065, -0.065, -0.065, ..., -0.36 , -0.355, -0.305],
       [-0.375, -0.375, -0.365, ..., -0.335, -0.33 , -0.325],
       ...,
       [ 0.055,  0.065,  0.075, ..., -0.11 , -0.125, -0.115],
       [-0.23 , -0.235, -0.23 , ..., -0.385, -0.39 , -0.395],
       [ 0.075,  0.06 ,  0.055, ...,  0.04 ,  0.035,  0.025]])

In [5]:
Y

array(['N', 'N', 'N', ..., 'N', 'N', 'N'], dtype='<U1')

In [6]:
df_x = pd.DataFrame(X)
df_x

,0,1,2,3,4,5,6,7,8,9,...,350,351,352,353,354,355,356,357,358,359
0,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.120,-0.135,...,-0.325,-0.330,-0.350,-0.365,-0.360,-0.380,-0.425,-0.445,-0.475,-0.510
1,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.080,-0.080,...,-0.235,-0.250,-0.270,-0.280,-0.290,-0.300,-0.330,-0.360,-0.355,-0.305
2,-0.375,-0.375,-0.365,-0.365,-0.360,-0.360,-0.365,-0.370,-0.355,-0.330,...,-0.315,-0.320,-0.315,-0.325,-0.335,-0.340,-0.335,-0.335,-0.330,-0.325
3,-0.330,-0.325,-0.315,-0.305,-0.305,-0.290,-0.290,-0.275,-0.270,-0.245,...,-0.190,-0.190,-0.200,-0.200,-0.195,-0.220,-0.205,-0.190,-0.200,-0.220
4,-0.355,-0.360,-0.355,-0.350,-0.340,-0.345,-0.345,-0.345,-0.345,-0.330,...,-0.385,-0.380,-0.370,-0.375,-0.385,-0.390,-0.395,-0.385,-0.385,-0.385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218983,0.160,0.140,0.125,0.110,0.090,0.085,0.085,0.085,0.070,0.065,...,-0.055,-0.060,-0.045,-0.050,-0.065,-0.080,-0.100,-0.095,-0.075,-0.060
218984,-0.335,-0.320,-0.320,-0.325,-0.330,-0.345,-0.350,-0.370,-0.360,-0.355,...,-0.200,-0.195,-0.200,-0.215,-0.215,-0.210,-0.200,-0.215,-0.225,-0.225
218985,0.055,0.065,0.075,0.065,0.070,0.050,0.040,0.045,0.045,0.060,...,-0.085,-0.100,-0.110,-0.125,-0.120,-0.105,-0.105,-0.110,-0.125,-0.115
218986,-0.230,-0.235,-0.230,-0.235,-0.225,-0.240,-0.230,-0.255,-0.235,-0.225,...,-0.385,-0.410,-0.420,-0.415,-0.405,-0.390,-0.390,-0.385,-0.390,-0.395


In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_x = scaler.fit_transform(df_x)
df_x = pd.DataFrame(df_x)


In [8]:
import pywt

def madev(d, axis=None):
    'Mean absolute deviation of a signal'
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

def wavelet_denoising(x, wavelet='sym4', level=1):
    coeff = pywt.wavedec(x, wavelet, mode="per")
    sigma = (1/0.6745) * madev(coeff[-level])
    uthresh = sigma * np.sqrt(2 * np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])
    return pywt.waverec(coeff, wavelet, mode='per')


df_x = wavelet_denoising(df_x, wavelet='sym4', level=2)
df_x = pd.DataFrame(df_x)

In [9]:
df_y = pd.DataFrame(Y, columns=['ColumnName'])
print(df_y)

       ColumnName
0               N
1               N
2               N
3               N
4               N
...           ...
218983          N
218984          N
218985          N
218986          N
218987          N

[218988 rows x 1 columns]


In [10]:
value_counts_y = df_y['ColumnName'].value_counts()
print(value_counts_y)

N    181262
Q     16086
V     14472
S      5562
F      1606
Name: ColumnName, dtype: int64


In [11]:
df_fusionné = pd.concat([df_y, df_x], axis=1)
df_fusionné

,ColumnName,0,1,2,3,4,5,6,7,8,...,350,351,352,353,354,355,356,357,358,359
0,N,0.462657,0.482548,0.484076,0.480509,0.483015,0.486312,0.486521,0.485864,0.486532,...,0.465025,0.465029,0.464824,0.464402,0.465701,0.466904,0.465382,0.464323,0.457096,0.448836
1,N,0.482615,0.482878,0.483020,0.482990,0.483850,0.485008,0.486124,0.487286,0.488439,...,0.481455,0.483832,0.481466,0.477827,0.472715,0.467783,0.468938,0.472220,0.476545,0.481245
2,N,0.466488,0.466541,0.466619,0.466730,0.466863,0.467018,0.467201,0.467403,0.467586,...,0.467761,0.467619,0.467456,0.467288,0.467140,0.467001,0.466853,0.466721,0.466592,0.466490
3,N,0.476074,0.475991,0.475932,0.475902,0.475899,0.475923,0.475974,0.476052,0.476144,...,0.478223,0.477955,0.477692,0.477437,0.477199,0.476973,0.476753,0.476549,0.476359,0.476191
4,N,0.463824,0.463999,0.464214,0.464463,0.464731,0.465019,0.465329,0.465651,0.465978,...,0.465537,0.464740,0.464300,0.464034,0.463862,0.463779,0.463687,0.463634,0.463640,0.463698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218983,N,0.500123,0.501216,0.502282,0.503381,0.504590,0.505775,0.507026,0.508180,0.508768,...,0.495130,0.495042,0.495292,0.495704,0.496254,0.496892,0.497418,0.497970,0.498541,0.499176
218984,N,0.472220,0.470893,0.469246,0.467733,0.467381,0.467319,0.467991,0.468719,0.467062,...,0.482340,0.483354,0.483058,0.482150,0.481007,0.479616,0.478215,0.476751,0.475234,0.473691
218985,N,0.493083,0.494277,0.495340,0.496525,0.498185,0.499842,0.501992,0.503957,0.503893,...,0.492201,0.492230,0.492237,0.492235,0.492392,0.492585,0.492499,0.492423,0.492293,0.492282
218986,N,0.466555,0.467796,0.468975,0.470227,0.471770,0.473322,0.475033,0.476623,0.477198,...,0.462120,0.462484,0.462790,0.463097,0.463529,0.464005,0.464356,0.464741,0.465110,0.465571


In [12]:
df_fusionné.to_csv('df_fusionné.csv', index=False)